## Setup  

In this notebook we extend the **channel–flow** example to **density-based topology optimisation (TO)** of a double pipe.  
For the forward Navier–Stokes–Brinkman implementation, refer to
`solve_fluid.ipynb`.

\[
\begin{aligned}
\min_{\gamma}\;\; & J
      = \int_{\Omega}
        \underbrace{\tfrac12(\,\mu\,\bigl(\nabla\mathbf u
        +\nabla\mathbf u^{\mathsf T}\bigr)\!:\!
        \bigl(\nabla\mathbf u+\nabla\mathbf u^{\mathsf T}\bigr) +\alpha(\gamma)\,\mathbf u\cdot\mathbf u)}_{\text{viscous dissipation}}
        \;d\Omega                                             \\[4pt]
\text{s.\,t.}\;\;
& \mathbf R(\gamma, \mathbf u, p) = \mathbf 0 \qquad\; \text{(steady NS–Brinkman)}\\[2pt]
& g(\gamma)=1-\frac{\displaystyle
             \sum_e \gamma_e\,v_e}{V^{\ast}} \le 0        \qquad
  \text{(volume / porosity constraint)}
\end{aligned}
\]

where  

* **Design field**  $\gamma_e \in [0,1]$ — element fill ratio (0 = fluid, 1 = solid)  
* **State variables**  $\mathbf u$ (velocity) and $p$ (pressure)  
* **Residual**  $\mathbf R(\gamma,\mathbf u,p)=\mathbf 0$ — discrete steady NS–Brinkman equations  
* $v_e$ — element area, $V^{\ast}$ — prescribed minimum solid/material fraction  

The objective $J$ minimizes **total viscous dissipation**
(equivalent to pressure drop for a fixed flow rate), a common goal in
fluid-device design.  
To target another metric (e.g.\ maximise flow rate or heat transfer)
simply replace the definition of $J$.

### Double Pipe Boundary Conditions
To illustrate fluid TO, we consider a simple double pipe problem, as shown in the figure below:

<img src="../../figures/double_pipe.png" alt="Channel" style="width: 500px;"/>

### Design parametrisation  

We adopt the standard **element-wise density method**: the design vector
$\gamma$ is mapped to the Brinkman damping coefficient
$\alpha(\gamma)$ via the RAMP scheme.

### Optimiser  

We employ the **Method of Moving Asymptotes (MMA)**.  
Gradients $\partial J/\partial\gamma$ and
$\partial g/\partial\gamma$ are produced automatically with JAX’s AD.

NOTE: We follow a convention similar to those found in the solid mechanics literature of Topology optimization where the solids are attributed with a pseudodensity of 1 and voids/fluids with a pseudodensity of 0. This is in contrast to literature in fluid mechanics where the vice-versa is often use. 

In [ ]:
import functools
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
import matplotlib.pyplot as plt
import numpy as np

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.utils as _utils
import toflux.src.bc as _bc
import toflux.src.fe_fluid as _fea
import toflux.src.solver as _solv
import toflux.src.mma as _mma
import toflux.src.viz as _viz

jax.config.update("jax_enable_x64", True)
plt.rcParams.update(_viz.high_res_plot_settings)

_Field = _fea.FluidField
_cmap = _viz.fluid_cmap

# Define Geometry and Mesh 

Below we build the **double_pipe** benchmark commonly used in fluid TO.

#### Geometry  
A simple double-pipe is described in a json and passed as the geometry. This geometry is passed to the mesher.

#### Discretisation  
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.  
* Both primary fields—velocity $(\mathbf u=(u_x,u_y))$ **and** pressure (p\)—are interpolated with the same Q1 shape functions (equal-order Q1/Q1 formulation).

#### Numerical integration  
Element contributions (mass, convection, brinkman, viscous and stability terms) are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.

In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/double_pipe.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=30,
  nely_desired=30,
  dofs_per_node=3,
  gauss_order=2,
)

_viz.plot_brep(geom)

### Material  

For this fluid demonstration we keep the parameters dimensionless:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1** |
| $(\mu)$ | Dynamic viscosity | **1** |

We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the design variable $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.

In [ ]:
mat_params = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)
min_mat_frac = 0.666
mat_frac = min_mat_frac * jnp.ones((mesh.num_elems,))

min_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 100.0 * mesh.bounding_box.lx
)
max_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 1.0e-2 * mesh.bounding_box.lx
)
init_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 1.0e-1 * mesh.bounding_box.lx
)


inv_permeability_ext = _utils.Extent(min=min_inv_permeability, max=max_inv_permeability)

brink_inter_factor = _mat.calculate_interpolation_factor(
  inv_permeability_ext, init_inv_permeability, min_mat_frac
)


# Brinkman Penalty Visualization

We visualize the Brinkman penalty here. The initial RAMP penalty, determined by `brink_inter_factor`, allows fluid flow in the domain at the beginning of the optimization. As the optimization proceeds, the penalty becomes more stricter, restricting fluid flow in the solid regions.
Note: Lower inverse permeability/brinkman penalty allows fluid flow.

In [ ]:
test_mat_frac = np.linspace(0.0, 1.0)
brinkman_penalty = _mat.compute_ramp_interpolation(
  prop=test_mat_frac,
  ramp_penalty=brink_inter_factor,
  prop_ext=inv_permeability_ext,
  mode="convex",
)
plt.figure(figsize=(8, 4))
plt.plot(test_mat_frac, brinkman_penalty, '-o', label='Brinkman Penalty')
plt.xlabel('Material Fraction')
plt.ylabel('Brinkman Penalty')
plt.title('Brinkman Penalty vs Material Fraction')
plt.legend()
plt.show()

# Boundray Conditions


| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet top and bottom(left vertical edge)** | Dirichlet | $(u = parabolic)$, $(v = 0)$ |
| **Outlet top and bottom (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{characteristic length}.
$$


In [ ]:
inlet_fraction = 1.0 / 6.0
char_length = inlet_fraction * mesh.bounding_box.ly
reynolds_num = 0.1667
char_velocity = reynolds_num * mat_params.kinematic_viscosity / (char_length)

In [ ]:
face_tol = mesh.elem_size[0] * 0.5

# inlet-top condition
inlet_top_faces = _bc.identify_faces(mesh, edges=[geom.edges[3]], tol=face_tol)
n_faces = len(inlet_top_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n_faces + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_Field.U_VEL, face_node_vals)
v_vel = (_Field.V_VEL, jnp.zeros_like(face_node_vals))
inlet_top_face_val = [u_vel, v_vel]

# inlet bottom condition
inlet_btm_faces = _bc.identify_faces(mesh, edges=[geom.edges[1]], tol=face_tol)
n_faces = len(inlet_btm_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n_faces + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_Field.U_VEL, face_node_vals)
v_vel = (_Field.V_VEL, jnp.zeros_like(face_node_vals))
inlet_btm_face_val = [u_vel, v_vel]

# wall condition
wall_faces = _bc.identify_faces(
  mesh,
  edges=[
    geom.edges[0],
    geom.edges[2],
    geom.edges[4],
    geom.edges[6],
    geom.edges[8],
    geom.edges[10],
    geom.edges[5],
    geom.edges[11],
  ],
  tol=face_tol,
)
n = len(wall_faces)
u_vel = (_Field.U_VEL, jnp.zeros(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]

# out flow condition
outflow_faces = _bc.identify_faces(
  mesh, edges=[geom.edges[7], geom.edges[9]], tol=face_tol
)
n = len(outflow_faces)
v_vel = (_Field.V_VEL, jnp.zeros(n))
pres = (_Field.PRESSURE, jnp.zeros(n))
outflow_face_val = [v_vel, pres]


inlet_top_bc = _bc.DirichletBC(
  elem_faces=inlet_top_faces, values=inlet_top_face_val, name="Inlet Top"
)
inlet_btm_bc = _bc.DirichletBC(
  elem_faces=inlet_btm_faces, values=inlet_btm_face_val, name="Inlet Bottom"
)
outflow_bc = _bc.DirichletBC(
  elem_faces=outflow_faces, values=outflow_face_val, name="Outflow"
)
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="Wall")


bcs_list = [
  wall_bc,
  inlet_top_bc,
  inlet_btm_bc,
  outflow_bc,
]

bc = _bc.process_boundary_conditions(bcs_list, mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_grid_mesh(mesh, ax=ax, colorbar=False)
_viz.plot_bc(bcs_list, mesh, ax=ax)
plt.show()


# Solver


The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'.

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "rtol": 1.0e-5,
    "scipy_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-6},
}


# Initialize

We initialize the FEA solver with the mesh, material, boundary conditions and solver settings. We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
flow_solver = _fea.FluidSolver(mesh, bc, mat_params, solver_settings=solver_settings)
init_press_vel = jnp.zeros((mesh.num_dofs,))
init_press_vel = init_press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

# Optimize

In [ ]:
@functools.partial(jax.jit, static_argnames=("flow_solver",))
def objective_function(
  mat_frac: jnp.ndarray,
  flow_solver: _fea.FluidSolver,
  ramp_penalty: float,
  init_press_vel=None,
):
  def objective_wrapper(mat_frac):
    brinkman_penalty = _mat.compute_ramp_interpolation(
      prop=mat_frac,
      ramp_penalty=ramp_penalty,
      prop_ext=inv_permeability_ext,
      mode="convex",
    )
    press_vel = _solv.modified_newton_raphson_solve(
      flow_solver, init_press_vel, brinkman_penalty
    )

    elem_obj = jax.vmap(flow_solver.compute_elem_dissipated_power)(
      brinkman_penalty, press_vel[mesh.elem_dof_mat], mesh.elem_node_coords
    )
    obj = jnp.sum(elem_obj)
    return obj, (press_vel, mat_frac)

  (obj, (press_vel, mat_frac)), d_obj = jax.value_and_grad(
    objective_wrapper, has_aux=True
  )(mat_frac)
  return obj, d_obj.reshape((-1, 1)), press_vel, mat_frac

# Constraint

We define a simple material constraint optimization problem where we restrict the minimum material/solid the design can assume. With the user defined `min_mat_frac` we compute the material constraint. Once again, the gradients are computed automatically via jax's `value_and_grad`.

In [ ]:
def constraint_function(mat_frac: ArrayLike, min_mat_frac: float):
  def constraint_wrapper(mat_frac):
    mean_mat_frac = jnp.mean(mat_frac)
    return 1.0 - (mean_mat_frac / min_mat_frac)

  mat_cons, d_mat_cons = jax.value_and_grad(constraint_wrapper)(mat_frac)
  return jnp.array([mat_cons]), d_mat_cons.T

# Optimize
Here, we define the optimization loop. We use the MMA optimizer. We begin the optimization with a uniform design corresponding to the minimum allowed material/solid fraction. The optimization flowchart is illustrated below:
<img src="../../figures/flowchart_fluid_TO.png" alt="flowchart" style="width: 800px;"/>

In [ ]:
def optimize_design(
  fe: _fea.FluidSolver,
  min_mat_frac: float,
  max_iter: int,
  move_limit: float = 1e-2,
  kkt_tol: float = 1e-3,
  step_tol: float = 1e-3,
  plot_interval: int = 5,
):
  design_var = min_mat_frac * np.ones((fe.mesh.num_elems, 1))
  num_design_var = design_var.shape[0]
  num_cons = 1
  lower_bound = np.zeros((num_design_var, 1))
  upper_bound = np.ones((num_design_var, 1))
  mma_params = _mma.MMAParams(
    max_iter=max_iter,
    kkt_tol=kkt_tol,
    step_tol=step_tol,
    move_limit=move_limit,
    num_design_var=num_design_var,
    num_cons=num_cons,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
  )
  mma_state = _mma.init_mma(design_var, mma_params)

  history = {"obj": [], "vol_cons": []}
  press_vel = jnp.zeros((fe.mesh.num_dofs,))
  press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])
  while not mma_state.is_converged:
    cont_param = min(20.0, 1.0 + 19.0 * (mma_state.epoch / 75.0) ** 2)
    ramp_penalty = brink_inter_factor / cont_param
    objective, grad_obj, press_vel, mat_frac = objective_function(
      mma_state.x, fe, ramp_penalty, press_vel
    )
    press_vel = jax.lax.stop_gradient(press_vel)
    press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])

    constr, grad_cons = constraint_function(mma_state.x, min_mat_frac)
    mma_state = _mma.update_mma(
      mma_state, mma_params, objective, grad_obj, constr, grad_cons
    )

    status = f"epoch {mma_state.epoch} J {objective:.2E} mc {constr[0]:.2F}"
    print(status)
    if mma_state.epoch % plot_interval == 0 or mma_state.epoch == 1:
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(
        mesh=mesh,
        field=mma_state.x.reshape(-1),
        ax=ax,
        colorbar=True,
        cmap=_cmap,
        val_range=(0.0, 1.0),
      )
      ax.set_title(status)
      press_vel_elem = press_vel[mesh.elem_dof_mat]
      press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
      u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
      v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
      vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=vel_elem_mag, ax=ax, colorbar=True)
      ax.set_title("Velocity Magnitude")

      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=press_elem, ax=ax, colorbar=True)
      ax.set_title("Pressure")
      plt.show()

  return mma_state, history

In [ ]:
mma_state, u = optimize_design(
  flow_solver, min_mat_frac=min_mat_frac, max_iter=100, move_limit=2.0e-1
)